In [ ]:
!pip install nashpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for nashpy: filename=nashpy-0.0.21-py3-none-any.whl size=15280 sha256=8e4ee33c4289362ac7c541e798d0b2c25a29c03c1e74ea6fe3d1da49336b8f41
  Stored in directory: /root/.cache/pip/wheels/02/08/62/cf4fa931e0a317d180936b266169a57f4bb4eb801465bbe8a1
Successfully built nashpy


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import random
import nashpy as nsp

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from PIL import Image, ImageOps
import glob
import time
import copy
import numpy as np
import cv2 as cv
import logging
from resnet_RGB import resnet18

In [ ]:
from torch.utils import data
def adverserial(sigma, gamma, l=1):
    f = np.exp(gamma)*np.log(sigma) + l*gamma**2 - l*sigma**2
    return f

def create_adversary(gamma=np.linspace(-7,7,30), sigma=np.random.uniform(low=0.1,high=7, size=30)):
    ad = np.zeros(shape=(len(sigma), len(gamma)))
    for i in range(len(sigma)):
        for j in range(len(gamma)):
            ad[i,j] = adverserial(sigma=sigma[i], gamma=gamma[j], l=1)
    
    return ad

def create_game(game_matrix, end_t=60):
    game = nsp.Game(game_matrix)
    start = time.time()
    equ = game.lemke_howson_enumeration()
    eq = []
    for e in equ:
        eq.append(e)
        end = time.time()
        if (end-start==end_t):
            return eq
            
    return eq

def solve_game(gamma=np.linspace(-5, 5,30), sigma=np.random.uniform(low=0.1,high=5, size=30)):
    eqb = create_game(create_adversary())
    veqb = []
    
    try:
        for e in eqb:
            if any(np.isnan(e[0])) or any(np.isnan(e[1])) :
                pass
            else:
                veqb.append(e)
 
    except:
        print('no equilibrium found')
        return 0
    
    cost= []
    for v in veqb:
        #find all non-zero probablities
        idx_as = np.where(v[0]!=0)
        idx_ag = np.where(v[1]!=0)
        
        #mixed policies
        prob_s = v[0][idx_as]
        prob_g = v[1][idx_ag]
        
        #actions
        a_s = sigma[idx_as]
        a_g = gamma[idx_ag]
        
        #discrete action weighted with the it mixed policy
        action_s = random.choices(a_s, weights=tuple(prob_s), k=1)
        action_g = random.choices(a_g, weights=tuple(prob_g), k=1)
 
        cost.append(adverserial(action_s[0], action_g[0],1))

    cost_min = min(cost)
    cost_max = max(cost)
    if abs(cost_min)>cost_max:
        return cost_min
    else:
        return cost_max


def ad_proc(PIL_image):
  cost = solve_game() #solve game for an random choice of sigma

  data_transforms = transforms.Compose([
        transforms.RandomResizedCrop((256,256)),
        # transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
  
  
  return torch.add(data_transforms(PIL_image), cost/40)

  



In [ ]:

def prepare_dataloader(num_workers=4, train_batch_size=128, eval_batch_size=256):

    data_dir=r'/content/drive/MyDrive/DataRedo/'
    

    train_set = datasets.ImageFolder(os.path.join(data_dir, "train"),
                                         ad_proc)

    train_loaders = torch.utils.data.DataLoader(train_set, 
                                                shuffle=True,
                                                batch_size=train_batch_size,
                                                num_workers=num_workers)
    
    val_set = datasets.ImageFolder(os.path.join(data_dir, "val"),
                                         ad_proc)
    val_loader = torch.utils.data.DataLoader(val_set, 
                                                shuffle=True,
                                                batch_size=eval_batch_size,
                                                num_workers=num_workers)

    return train_loaders, val_loader

def prepare_dataloader(num_workers=4, train_batch_size=128, eval_batch_size=256):
    data_dir=r'/content/drive/MyDrive/DataRedo/'

    data_transforms = {
        'train': transforms.Compose([
            transforms.RandomResizedCrop((256,256)),
            # transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'val': transforms.Compose([
            transforms.Resize((256,256)),
            # transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }


    train_set = datasets.ImageFolder(os.path.join(data_dir, "train"),
                                          data_transforms["train"])
    train_loaders = torch.utils.data.DataLoader(train_set, 
                                                shuffle=True,
                                                batch_size=train_batch_size,
                                                num_workers=num_workers)
    
    val_set = datasets.ImageFolder(os.path.join(data_dir, "val"),
                                          data_transforms["val"])
    
    val_loaders = torch.utils.data.DataLoader(val_set,
                                              shuffle=True,
                                              batch_size=eval_batch_size,
                                              num_workers=num_workers)
    
    return train_loaders, val_loaders

In [ ]:
def set_random_seeds(random_seed=0):

    torch.manual_seed(random_seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)
    random.seed(random_seed)
 

def evaluate_model(model, test_loader, device, criterion=None):

    model.eval()
    model.to(device)

    running_loss = 0
    running_corrects = 0
    class_names = ['Dog', 'Human', 'Deer', 'Cat']
    # class_names = [name[37:] for name in glob.glob(r'/content/drive/MyDrive/DataRedo/train/*')]
    class_names = dict(zip(range(0,len(class_names)), class_names))
    
    for inputs, labels in test_loader:

        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        if criterion is not None:
            loss = criterion(outputs, labels).item()
        else:
            loss = 0

        # statistics
        running_loss += loss * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    eval_loss = running_loss / len(test_loader.dataset)
    eval_accuracy = running_corrects / len(test_loader.dataset)

    return eval_loss, eval_accuracy

def write_checkpoint(model, optimizer, epoch, scheduler):
  state = {'epoch': epoch + 1, 'state': model.state_dict(),
             'optimizer': optimizer.state_dict(), 'scheduler': scheduler.state_dict()}
  filename = '/content/model_'
  
  torch.save(state, filename + f'CP_epoch{epoch + 1}.pth',_use_new_zipfile_serialization=False)


def load_checkpoint(model, optimizer, scheduler, filename='/content/model_CP_epoch11.pth'):
    # Note: Input model & optimizer should be pre-defined.  This routine only updates their states.
    start_epoch = 0
    if os.path.isfile(filename):
        print("=> loading checkpoint '{}'".format(filename))
        checkpoint = torch.load(filename)
        start_epoch = checkpoint['epoch']
        model.load_state_dict(checkpoint['state'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        scheduler.load_state_dict(checkpoint['scheduler'])
        print("=> loaded checkpoint '{}' (epoch {})"
                  .format(filename, checkpoint['epoch']))
        
        
    else:
        print("=> no checkpoint found at '{}'".format(filename))

    return model, optimizer, start_epoch, scheduler

def train_model(model, train_loader, test_loader, device, learning_rate=1e-1, num_epochs=200):

    # The training configurations were not carefully selected.

    criterion = nn.CrossEntropyLoss()

    model.to(device)

    # It seems that SGD optimizer is better than Adam optimizer for ResNet18 training on CIFAR10.
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
    # scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=500)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[65, 75, 85], gamma=0.75, last_epoch=-1)
    # optimizer = optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

    # Evaluation
    model.eval()
    eval_loss, eval_accuracy = evaluate_model(model=model, test_loader=test_loader, device=device, criterion=criterion)
    print("Epoch: {:02d} Eval Loss: {:.3f} Eval Acc: {:.3f}".format(-1, eval_loss, eval_accuracy))

    load_model = input('Load a model?')
    
    start_epoch=0
    if load_model:
      model, optimizer, start_epoch, scheduler = load_checkpoint(model=model, scheduler=scheduler, optimizer=optimizer)
    for epoch in range(start_epoch, num_epochs):

        if  epoch%10 == 0:
          write_checkpoint(model=model, epoch=epoch, scheduler=scheduler, optimizer=optimizer)          
          # model, optimizer, epoch, scheduler = load_checkpoint(model=model, scheduler=scheduler, optimizer=optimizer)    
          
          for state in optimizer.state.values():
            for k, v in state.items():
                if isinstance(v, torch.Tensor):
                    state[k] = v.to(device)
        # Training
        model.train()

        running_loss = 0
        running_corrects = 0

        for inputs, labels in train_loader:
            inputs = torch.FloatTensor(inputs)
            inputs = inputs.to(device)
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        train_loss = running_loss / len(train_loader.dataset)
        train_accuracy = running_corrects / len(train_loader.dataset)

        # Evaluation
        model.eval()
        eval_loss, eval_accuracy = evaluate_model(model=model, test_loader=test_loader, device=device, criterion=criterion)

        # Set learning rate scheduler
        scheduler.step()

        print("Epoch: {:03d} Train Loss: {:.3f} Train Acc: {:.3f} Eval Loss: {:.3f} Eval Acc: {:.3f}".format(epoch, train_loss, train_accuracy, eval_loss, eval_accuracy))

    return model

def calibrate_model(model, loader, device=torch.device("cpu:0")):

    model.to(device)
    model.eval()
    
    for inputs,labels in loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        _ = model(inputs)

def measure_inference_latency(model,
                              device,
                              input_size=(1, 3, 32, 32),
                              num_samples=100,
                              num_warmups=10):

    model.to(device)
    model.eval()

    x = torch.rand(size=input_size).to(device)

    with torch.no_grad():
        for _ in range(num_warmups):
            _ = model(x)
    torch.cuda.synchronize()

    with torch.no_grad():
        start_time = time.time()
        for _ in range(num_samples):
            _ = model(x)
            torch.cuda.synchronize()
        end_time = time.time()
    elapsed_time = end_time - start_time
    elapsed_time_ave = elapsed_time / num_samples

    return elapsed_time_ave

def save_model(model, model_dir, model_filename):

    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    model_filepath = os.path.join(model_dir, model_filename)
    torch.save(model.state_dict(), model_filepath)

def load_model(model, model_filepath, device):

    model.load_state_dict(torch.load(model_filepath, map_location=device))

    return model

def save_torchscript_model(model, model_dir, model_filename):

    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    model_filepath = os.path.join(model_dir, model_filename)
    torch.jit.save(torch.jit.script(model), model_filepath)

def load_torchscript_model(model_filepath, device):

    model = torch.jit.load(model_filepath, map_location=device)

    return model

def create_model(num_classes=10):

    # The number of channels in ResNet18 is divisible by 8.
    # This is required for fast GEMM integer matrix multiplication.
    # model = torchvision.models.resnet18(pretrained=False)
    model = resnet18(num_classes=num_classes, pretrained=False)

    # We would use the pretrained ResNet18 as a feature extractor.
    # for param in model.parameters():
    #     param.requires_grad = False
    
    # Modify the last FC layer
    # num_features = model.fc.in_features
    # model.fc = nn.Linear(num_features, 10)

    return model

class QuantizedResNet18(nn.Module):
    def __init__(self, model_fp32):
        super(QuantizedResNet18, self).__init__()
        # QuantStub converts tensors from floating point to quantized.
        # This will only be used for inputs.
        self.quant = torch.quantization.QuantStub()
        # DeQuantStub converts tensors from quantized to floating point.
        # This will only be used for outputs.
        self.dequant = torch.quantization.DeQuantStub()
        # FP32 model
        self.model_fp32 = model_fp32

    def forward(self, x):
        # manually specify where tensors will be converted from floating
        # point to quantized in the quantized model
        x = self.quant(x)
        x = self.model_fp32(x)
        # manually specify where tensors will be converted from quantized
        # to floating point in the quantized model
        x = self.dequant(x)
        return x

def model_equivalence(model_1, model_2, device, rtol=1e-05, atol=1e-08, num_tests=100, input_size=(1,3,32,32)):

    model_1.to(device)
    model_2.to(device)

    for _ in range(num_tests):
        x = torch.rand(size=input_size).to(device)
        y1 = model_1(x).detach().cpu().numpy()
        y2 = model_2(x).detach().cpu().numpy()
        if np.allclose(a=y1, b=y2, rtol=rtol, atol=atol, equal_nan=False) == False:
            print("Model equivalence test sample failed: ")
            print(y1)
            print(y2)
            return False

    return True


In [ ]:
import torch
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
torch.cuda.empty_cache()

In [ ]:
def main():

  ########################Intialization#########################################

    # random_seed = 42
    num_classes = 4
    cuda_device = torch.device("cuda:0")
    cpu_device = torch.device("cpu:0")


    model_dir = "saved_models"
    model_filename = "RGB_RedoData_resnet18.pt"
    # quantized_model_filename = "resnet18_quantized_spatial_pyramid.pt"
    # model_filepath = os.path.join(model_dir, model_filename)
    model_filepath = model_filename
    # quantized_model_filepath = os.path.join(model_dir, quantized_model_filename)
  
    # set_random_seeds(random_seed=random_seed)

    #Create an instance of sift
    # sift = cv.xfeatures2d.SIFT_create(nOctaveLayers=12, edgeThreshold=20, sigma=1.4)

    # Create an untrained model.
    model = create_model(num_classes=num_classes)
    
    train_loader, test_loader = prepare_dataloader(num_workers=4, train_batch_size=16, eval_batch_size=32)
    
    ###################################Training of model END#####################
    print("Training Model...")
    
    model = train_model(model=model, train_loader=train_loader, test_loader=test_loader, device=cuda_device, learning_rate=1e-1, num_epochs=80)

    
    model = train_model(model=model, train_loader=train_loader, test_loader=test_loader, device=cuda_device, learning_rate=1e-1, num_epochs=80)
    # # Save model.
    save_model(model=model, model_dir=model_dir, model_filename=model_filename)
    # Load a pretrained model.
    
if __name__ == "__main__":
    torch.cuda.empty_cache()
    main()

Training Model...


/usr/local/lib/python3.7/dist-packages/nashpy/integer_pivoting/integer_pivoting.py:54: RuntimeWarning: overflow encountered in multiply
  tableau[i, :] * pivot_element
/usr/local/lib/python3.7/dist-packages/nashpy/integer_pivoting/integer_pivoting.py:55: RuntimeWarning: overflow encountered in multiply
  - tableau[pivot_row_index, :] * tableau[i, column_index]
/usr/local/lib/python3.7/dist-packages/nashpy/integer_pivoting/integer_pivoting.py:55: RuntimeWarning: invalid value encountered in subtract
  - tableau[pivot_row_index, :] * tableau[i, column_index]
/usr/local/lib/python3.7/dist-packages/nashpy/integer_pivoting/integer_pivoting.py:31: RuntimeWarning: invalid value encountered in true_divide
  return np.argmax(tableau[:, column_index] / tableau[:, -1])
/usr/local/lib/python3.7/dist-packages/nashpy/integer_pivoting/integer_pivoting.py:54: RuntimeWarning: invalid value encountered in multiply
  tableau[i, :] * pivot_element
/usr/local/lib/python3.7/dist-packages/nashpy/integer_pivo

Epoch: -1 Eval Loss: 1.892 Eval Acc: 0.254


/usr/local/lib/python3.7/dist-packages/nashpy/integer_pivoting/integer_pivoting.py:54: RuntimeWarning: overflow encountered in multiply
  tableau[i, :] * pivot_element
/usr/local/lib/python3.7/dist-packages/nashpy/integer_pivoting/integer_pivoting.py:55: RuntimeWarning: overflow encountered in multiply
  - tableau[pivot_row_index, :] * tableau[i, column_index]
/usr/local/lib/python3.7/dist-packages/nashpy/integer_pivoting/integer_pivoting.py:54: RuntimeWarning: overflow encountered in multiply
  tableau[i, :] * pivot_element
/usr/local/lib/python3.7/dist-packages/nashpy/integer_pivoting/integer_pivoting.py:55: RuntimeWarning: invalid value encountered in subtract
  - tableau[pivot_row_index, :] * tableau[i, column_index]
/usr/local/lib/python3.7/dist-packages/nashpy/integer_pivoting/integer_pivoting.py:55: RuntimeWarning: overflow encountered in multiply
  - tableau[pivot_row_index, :] * tableau[i, column_index]
/usr/local/lib/python3.7/dist-packages/nashpy/integer_pivoting/integer_piv

Epoch: 000 Train Loss: 1.673 Train Acc: 0.311 Eval Loss: 1.439 Eval Acc: 0.285


...

/usr/local/lib/python3.7/dist-packages/nashpy/integer_pivoting/integer_pivoting.py:54: RuntimeWarning: overflow encountered in multiply
  tableau[i, :] * pivot_element
/usr/local/lib/python3.7/dist-packages/nashpy/integer_pivoting/integer_pivoting.py:55: RuntimeWarning: overflow encountered in multiply
  - tableau[pivot_row_index, :] * tableau[i, column_index]
/usr/local/lib/python3.7/dist-packages/nashpy/integer_pivoting/integer_pivoting.py:54: RuntimeWarning: overflow encountered in multiply
  tableau[i, :] * pivot_element
/usr/local/lib/python3.7/dist-packages/nashpy/integer_pivoting/integer_pivoting.py:55: RuntimeWarning: invalid value encountered in subtract
  - tableau[pivot_row_index, :] * tableau[i, column_index]
/usr/local/lib/python3.7/dist-packages/nashpy/integer_pivoting/integer_pivoting.py:55: RuntimeWarning: overflow encountered in multiply
  - tableau[pivot_row_index, :] * tableau[i, column_index]
/usr/local/lib/python3.7/dist-packages/nashpy/integer_pivoting/integer_piv

Epoch: 012 Train Loss: 1.429 Train Acc: 0.322 Eval Loss: 10.278 Eval Acc: 0.246


/usr/local/lib/python3.7/dist-packages/nashpy/integer_pivoting/integer_pivoting.py:54: RuntimeWarning: overflow encountered in multiply
  tableau[i, :] * pivot_element
/usr/local/lib/python3.7/dist-packages/nashpy/integer_pivoting/integer_pivoting.py:55: RuntimeWarning: overflow encountered in multiply
  - tableau[pivot_row_index, :] * tableau[i, column_index]
/usr/local/lib/python3.7/dist-packages/nashpy/integer_pivoting/integer_pivoting.py:55: RuntimeWarning: invalid value encountered in subtract
  - tableau[pivot_row_index, :] * tableau[i, column_index]
/usr/local/lib/python3.7/dist-packages/nashpy/integer_pivoting/integer_pivoting.py:31: RuntimeWarning: invalid value encountered in true_divide
  return np.argmax(tableau[:, column_index] / tableau[:, -1])
/usr/local/lib/python3.7/dist-packages/nashpy/integer_pivoting/integer_pivoting.py:54: RuntimeWarning: invalid value encountered in multiply
  tableau[i, :] * pivot_element
/usr/local/lib/python3.7/dist-packages/nashpy/integer_pivo

Epoch: 013 Train Loss: 1.386 Train Acc: 0.332 Eval Loss: 3.789 Eval Acc: 0.265


/usr/local/lib/python3.7/dist-packages/nashpy/integer_pivoting/integer_pivoting.py:54: RuntimeWarning: overflow encountered in multiply
  tableau[i, :] * pivot_element
/usr/local/lib/python3.7/dist-packages/nashpy/integer_pivoting/integer_pivoting.py:55: RuntimeWarning: overflow encountered in multiply
  - tableau[pivot_row_index, :] * tableau[i, column_index]
/usr/local/lib/python3.7/dist-packages/nashpy/integer_pivoting/integer_pivoting.py:54: RuntimeWarning: overflow encountered in multiply
  tableau[i, :] * pivot_element
/usr/local/lib/python3.7/dist-packages/nashpy/integer_pivoting/integer_pivoting.py:55: RuntimeWarning: invalid value encountered in subtract
  - tableau[pivot_row_index, :] * tableau[i, column_index]
/usr/local/lib/python3.7/dist-packages/nashpy/integer_pivoting/integer_pivoting.py:54: RuntimeWarning: overflow encountered in multiply
  tableau[i, :] * pivot_element
/usr/local/lib/python3.7/dist-packages/nashpy/integer_pivoting/integer_pivoting.py:55: RuntimeWarning

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Non SIFT Detection

In [ ]:
import torch
from torchvision.utils import draw_bounding_boxes
from torchvision.io import read_image
import torchvision
from torchvision import transforms
from PIL import Image
cuda_device = torch.device("cuda:0")
class_names= ['Cat', 'Deer', 'Dog', 'Human', 'Owl', 'Racoon']
    
model = torch.load('/content/adverserial_model.pt', map_location=cuda_device)

img_cat = Image.open("/content/cat.jpg").convert('RGB')
preprocess = transforms.Compose([
        transforms.Resize(256,256),
        transforms.ToTensor(),
        transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )])
#
# Pass the image for preprocessing and the image preprocessed
#
img_cat_preprocessed = preprocess(img_cat)
#
# Reshape, crop, and normalize the input tensor for feeding into network for evaluation
#
batch_img_cat_tensor = torch.unsqueeze(img_cat_preprocessed, 0)


model.eval()

out = model(batch_img_cat_tensor)
_, index = torch.max(out, 1)

#
# Find the score in terms of percentage by using torch.nn.functional.softmax function
# which normalizes the output to range [0,1] and multiplying by 100
#
percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
#
# Print the name along with score of the object identified by the model
#
print(class_names[index[0]], percentage[index[0]].item())
#
# Print the top 5 scores along with the image label. Sort function is invoked on the torch to sort the scores.
#
_, indices = torch.sort(out, descending=True)
[(class_names[idx], percentage[idx].item()) for idx in indices[0][:5]]

# bounding box in (xmin, ymin, xmax, ymax) format
bbox1 = [30, 45, 200, 200]
bbox = [bbox1]
bbox = torch.tensor(bbox, dtype=torch.int)
labels = class_names
pred_label = [class_names[idx] for idx in indices[0][:1]]
img = read_image("/content/cat.jpg")
img=draw_bounding_boxes(img, bbox,width=3,labels= pred_label,colors=[(255,0,0),(0,255,0)],fill =True,font_size=20)
img = torchvision.transforms.ToPILImage()(img)
img